#### Get VIIRS perimeters

In [ ]:
import datetime as dt
import geopandas as gpd
from owslib.ogcapi.features import Features
import pandas as pd
import geopandas as gpd

def get_fire_perimeters(bbox, start_date, end_date, min_area=None, min_duration=None):
    """
    Retrieve fire perimeters for a given region and time period.
    
    Parameters:
    bbox (list): Bounding box coordinates [west, south, east, north]
    start_date (str): Start date in format 'YYYY-MM-DD'
    end_date (str): End date in format 'YYYY-MM-DD'
    min_area (float): Minimum fire area in km² (optional)
    min_duration (int): Minimum fire duration in days (optional)
    
    Returns:
    GeoDataFrame: Fire perimeter data
    """
    # Format dates for API
    start_dt = f"{start_date}T00:00:00+00:00"
    end_dt = f"{end_date}T23:59:59+00:00"
    
    # Connect to API
    w = Features(url="https://firenrt.delta-backend.com")
    
    # Build filter string
    filters = []
    if min_area:
        filters.append(f"farea>{min_area}")
    if min_duration:
        filters.append(f"duration>{min_duration}")
    filter_str = " AND ".join(filters) if filters else None
    
    # Determine which collection to use based on dates
    current_year = dt.datetime.now().year
    start_year = int(start_date[:4])
    
    if start_year == current_year:
        collection = "public.eis_fire_lf_perimeter_nrt"
    elif 2018 <= start_year <= 2021:
        collection = "public.eis_fire_lf_perimeter_archive"
    else:
        raise ValueError("Data only available for years 2018-2021 and current year")
    
    # Query API
    results = w.collection_items(
        collection,
        bbox=[str(x) for x in bbox],
        datetime=[f"{start_dt}/{end_dt}"],
        limit=3000,
        filter=filter_str
    )
    
    # Convert to GeoDataFrame
    if results['numberMatched'] == 0:
        return gpd.GeoDataFrame()
        
    perimeters = gpd.GeoDataFrame.from_features(results["features"])
    perimeters = perimeters.set_crs("epsg:4326")
    perimeters = perimeters.sort_values(by="t")
    
    return perimeters

def plot_perimeters(gdf, base_map=True):
    """
    Plot fire perimeters on an interactive map
    
    Parameters:
    gdf (GeoDataFrame): Fire perimeter data
    base_map (bool): Whether to create a new map or add to existing
    
    Returns:
    folium.Map: Interactive map
    """
    if base_map:
        m = gdf.explore(
            style_kwds={"fillOpacity": 0.1},
            column="farea",
            tooltip=["fireid", "farea", "t"],
            zoom_start=8
        )
    else:
        m = gdf.explore(m=m, style_kwds={"fillOpacity": 0.1})
    return m

In [ ]:
def collect_fire_perimeters(bbox, start_year, end_year):
    """
    Collect fire perimeters for Western US by iterating month by month.
    Returns a single GeoDataFrame with all results.
    
    Parameters:
    bbox (list): [west, south, east, north] coordinates
    start_year (int): Starting year
    end_year (int): Ending year (inclusive)
    
    Returns:
    GeoDataFrame: Combined fire perimeter data
    """
    import pandas as pd
    import geopandas as gpd
    from datetime import datetime
    
    all_perims = []
    
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            start_date = f"{year}-{month:02d}-01"
            
            # Calculate end date accounting for different month lengths
            if month == 12:
                next_month = f"{year + 1}-01-01"
            else:
                next_month = f"{year}-{month + 1:02d}-01"
                
            try:
                perims = get_fire_perimeters(
                    bbox=bbox,
                    start_date=start_date,
                    end_date=next_month,
                    min_area=5
                )
                
                if not perims.empty:
                    all_perims.append(perims)
                    print(f"Collected data for {start_date}")
                    
            except Exception as e:
                print(f"Error collecting data for {start_date}: {str(e)}")
                continue
    
    if not all_perims:
        raise ValueError("No fire perimeter data collected")
        
    # Combine all monthly data
    combined_perims = pd.concat(all_perims, ignore_index=True)
    
    # Remove potential duplicates based on geometry and date
    combined_perims = combined_perims.drop_duplicates(subset=['geometry'])#, 'duration'])
    
    return combined_perims

In [ ]:
# Western US fires
# usw_bbox = [-125.551, 31.0529, -101.689, 49.439]  # [west, south, east, north]
# Montana fires
# -116.158447,44.386692,-103.963623,49.030665
mt_bbox = [-116.158447, 44.386692, -103.963623, 49.030665]
perims = get_fire_perimeters(
    bbox=mt_bbox,
    start_date="2019-06-01",
    end_date="2019-06-30",
    min_area=5  # fires larger than 5 km²
)

if not perims.empty:
    # m = plot_perimeters(perims[perims['fireid'] == 'F5831'])
    # display(m)
    
    # Print summary statistics
    print(f"\nFound {len(perims)} fire perimeters")
    print("\nLargest fires:")
    print(perims.sort_values('farea', ascending=False)[['fireid', 'farea', 't']].head())

In [ ]:
mt_fires_2019 = collect_fire_perimeters(mt_bbox, 2019, 2019)

In [ ]:
mt_fires_2019.fireid.value_counts()

----

In [ ]:
firemt_subset = mt_fires_2019[mt_fires_2019['fireid'] == 'F8212']

In [ ]:
firemt_subset

In [ ]:
# plot firemt_subset with osm basemap
m = plot_perimeters(firemt_subset)
display(m)